In [2]:
import pandas as pd
import numpy as np

from sklearn.metrics import accuracy_score

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.linear_model import LogisticRegression # Meta learner

In [3]:
df = pd.read_csv('data/bank_data_processed.csv')
df.head()

,Age,Income,Family,Education,Mortgage,Securities Account,CD Account,Online,CreditCard
0,34,180,1,3,0,0,0,0,0
1,38,130,4,3,134,0,0,0,0
2,46,193,2,3,0,0,0,0,0
3,38,119,1,2,0,0,1,1,1
4,42,141,3,3,0,1,1,1,0


In [4]:
X = df.drop(columns='CreditCard')
y = df.CreditCard

In [5]:
X_train, X_val, X_test = np.split(X, [int(.7*len(X)), int(.9*len(X))])
y_train, y_val, y_test = np.split(y, [int(.7*len(y)), int(.9*len(y))])

In [6]:
X_train.shape, X_val.shape, X_test.shape, y_train.shape, y_val.shape, y_test.shape

((336, 8), (96, 8), (48, 8), (336,), (96,), (48,))

In [7]:
clf1 = KNeighborsClassifier(n_neighbors=10)
clf2 = RandomForestClassifier(n_estimators=100)
clf3 = GaussianNB()
for clf in [clf1, clf2, clf3]:
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_val)
    print(clf.__class__.__name__, accuracy_score(y_val, y_pred))

KNeighborsClassifier 0.7083333333333334
RandomForestClassifier 0.8125
GaussianNB 0.8229166666666666


In [8]:
def get_predictions(x, y):

    pred_result = pd.DataFrame()

    i = 1

    for clf in [clf1, clf2, clf3]:
        y_pred = clf.predict(x)
        pred_result.insert(i-1, 'y_pred_' + str(i), y_pred)
        i += 1
    
    return pred_result

In [9]:
pred_result = get_predictions(X_val, y_val)
pred_result.head()

,y_pred_1,y_pred_2,y_pred_3
0,0,0,0
1,0,0,0
2,0,0,0
3,0,0,0
4,0,0,0


In [10]:
X_stack_train = pred_result
y_stack_train = y_val

In [11]:
clf_stack = LogisticRegression(solver='lbfgs', C=1, max_iter=200)
clf_stack.fit(X_stack_train, y_stack_train)

LogisticRegression(C=1, max_iter=200)

In [12]:
pred_result_test = get_predictions(X_test, y_test)
y_stack_pred = clf_stack.predict(pred_result_test)
print(f'Accuracy of stacking: {accuracy_score(y_test, y_stack_pred)}')

Accuracy of stacking: 0.8541666666666666
